# Prueba visual de función de eficiencias-Ignorar

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle

In [2]:
df = pd.read_csv("Datos\TomaPrueba", delim_whitespace=True, skiprows=9, names=["Brd", "Ch", "LG", "HG", "Tstamp_us", "TrgID", "NHits"], engine="python")
df

,Brd,Ch,LG,HG,Tstamp_us,TrgID,NHits
0,0,0,2488,4044,13373.784,0.0,64.0
1,0,1,3299,4018,NaN,NaN,NaN
2,0,2,3925,4071,NaN,NaN,NaN
3,0,3,4082,4077,NaN,NaN,NaN
4,0,4,4090,4074,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1275,0,59,50,53,NaN,NaN,NaN
1276,0,60,50,50,NaN,NaN,NaN
1277,0,61,50,55,NaN,NaN,NaN
1278,0,62,56,53,NaN,NaN,NaN


In [ ]:
#Funcion extra para graficar
def graficar(lista):    
    fig1, ax1 = plt.subplots(1, 1, figsize=(12, 6))
    ax1.set_xticks(range(32, 63, 2))  # Eje X de 0 a 10 con paso de 1
    ax1.set_yticks(range(0, 31, 2))  # Eje Y de -1 a 1 con paso de 1
    
    for i in lista:
    
        if i<=28:
            ax1.axhline(y=i, color='g', linewidth=2)
            ax1.axhline(y=i, color='g', linewidth=2)
        elif i>=32:
            ax1.axvline(x=i, color='r', linewidth=2)
            ax1.axvline(x=i, color='r', linewidth=2)
            
    fig1.savefig("grafico.png")
    

# funcion secundaria para discriminar eventos reales para activaciones triples y cudruples con adyacencia 
def conteo(df, cuadru, tripl, count, n_counts): # Dataframe, cuadruples, triples, lista de canales activados local, lista número de canales activados global, HG/LG, i-ésimo disparo
    
    graficar(count)
    
    n = len(count) # núm. canales activados
    #n_counts.append(n)
    
    # Discrimina rondas con cuadruples y que esten juntos cada pareja
    if n == 4:
        if count[0] == count[1] -1 and count[2] == count[3] - 1 and count[1]<=28 and count[2]>=32:
        
            # Parado desde cada canal, nota que él mismo, uno contiguo y otro 2 en el otro eje
            cuadru[count[0]] += 1
            cuadru[count[1]] += 1
            cuadru[count[2]] += 1
            cuadru[count[3]] += 1
            
            n_counts.append(n)
            
        #else:
            #error_count.append(count)
            
    # Discrimina rondas de triples
    elif n==3:
        # Discrimina si los canales están pegados en el eje X y el tercero pertenece al eje y
        if count[0] == count[1] - 1 and count[1] != count[2] - 1 and (count[1] <= 28 and count[2] >= 32):
            
            # Son triples aquellos contiguos al canal en y al no tener pareja
            # Condicionar para problema de bordes
            if count[2] > 32:
                tripl[count[2] -1] += 1
            if count[2] < 60:
                tripl[count[2] +1] += 1

            n_counts.append(n)
            
        # Discrimina si los canales están pegados en el eje Y y el primero pertenece al eje x
        elif count[0] != count[1] - 1 and count[1] == count[2] - 1 and (count[0] <= 28 and count[2] >= 32):
            
            # Son triples aquellos contiguos al canal en x al no tener pareja
            # Condicionar para problema de bordes
            if count[0] > 0:
                tripl[count[0] -1] += 1
            if count[0] < 28:
                tripl[count[0] +1] += 1
                
            n_counts.append(n)
        
        #else:
            #error_count.append(count)
    
    print("conteos: ", n_counts)
    print("cuadruples: ", cuadru)
    print("triples: ", tripl)
    A = True
    
    return cuadru, tripl, n_counts


def eficiencias(df, td_hg, td_lg):
    
    # Diccionarios para 3-activaciones y 4-activaciones {canal:contador} para HG y LG
    triples_h = {}
    cuadruples_h = {}
    triples_l = {}
    cuadruples_l = {}
    triples_b = {}
    cuadruples_b = {}

    # Conteos totales por disparo {Núm.canales:contador}
    conteo_total_h = {}
    conteo_total_l = {}
    conteo_total_b = {}

    # Se llenan los disccionarios
    for i in range(64):
        triples_h[i]= 0
        cuadruples_h[i] = 0
        triples_l[i]= 0
        cuadruples_l[i] = 0
        triples_b[i]= 0
        cuadruples_b[i] = 0

        conteo_total_h[i] = 0
        conteo_total_l[i] = 0
        conteo_total_b[i] = 0
    
    # Tambien se pueden activar 64 canales
    conteo_total_h[64] = 0
    conteo_total_l[64] = 0
    conteo_total_b[64] = 0
    
    # Lista de cantidad de canales activados por disparo con HG y LG
    n_counts_h= [] 
    n_counts_l= [] 
    n_counts_b= [] 
    
    # Recorre cada ronda de activación - i-ésima ronda
    for i in range(int(len(df)/64)):
        
        # Lista de los canales activos para aquellos discriminados en funcion de el HG, LG o b-ambos
        hg_count = []
        lg_count = []
        b_count = []
        
        # Recorre todos los canales de la ronda de activación - j-ésimo canal de la i-ésimo disparo
        for j in range(64):
            
            # Valor del H/L gain
            hg = df["HG"].values[i*64 + j]
            lg = df["LG"].values[i*64 + j]

            # Discrimina y agrega el canal a la lista de canales activados
            if hg > td_hg and lg > td_lg:
                hg_count.append(j)
                lg_count.append(j)
                b_count.append(j)
            else:
                if hg > td_hg:
                    hg_count.append(j)
                if lg > td_lg:
                    lg_count.append(j)
        
        print("high count: ", hg_count)
        print("low count: ", lg_count)
        print("both count: ", b_count)
        
        # Agrega cantidad de canales activados en el disparo
        conteo_total_h[len(hg_count)] += 1
        conteo_total_l[len(lg_count)] += 1
        conteo_total_b[len(b_count)] += 1
        
        conteo_total = [conteo_total_h,conteo_total_l,conteo_total_b]
        
        # Para cada tipo HG/LG/Both se determinará los conteos cuadruples y triples del i-ésimo disparo        
        print("high gain")
        datos_h = conteo(df, cuadruples_h, triples_h, hg_count, n_counts_h) 
        print("low gain")
        datos_l = conteo(df, cuadruples_l, triples_l, lg_count, n_counts_l)
        print("both gain")
        datos_b = conteo(df, cuadruples_b, triples_b, b_count, n_counts_b)
        
        
    return datos_h, datos_l, datos_b, conteo_total


td_hg, td_lg = 500, 200

datos_h, datos_l, datos_b, conteo_total = eficiencias(df, td_hg, td_lg)
cuadruples_h, triples_h, n_counts_h = datos_h
cuadruples_l, triples_l, n_counts_l = datos_l
cuadruples_b, triples_b, n_counts_b = datos_b
conteo_total_h, conteo_total_l, conteo_total_b = conteo_total

high count:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 63]
low count:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 62, 63]
both count:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 63]
high gain
conteos:  []
cuadruples:  {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 

C:\Users\danie\AppData\Local\Temp\ipykernel_13008\2482594477.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig1, ax1 = plt.subplots(1, 1, figsize=(12, 6))


conteos:  [3, 3, 3, 4]
cuadruples:  {0: 0, 1: 0, 2: 0, 3: 1, 4: 1, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0, 51: 0, 52: 0, 53: 0, 54: 0, 55: 0, 56: 0, 57: 0, 58: 0, 59: 0, 60: 0, 61: 0, 62: 0, 63: 0}
triples:  {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 1, 36: 1, 37: 1, 38: 0, 39: 0, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0, 51: 0, 52: 1, 53: 0, 54: 1, 55: 0, 56: 0, 57: 0, 58: 0, 59: 0, 60: 0, 61: 0, 62: 0, 63: 0}
high count:  [7, 8, 9, 10]
low count:  [7, 8, 9, 10]
both count:  [7, 8, 9, 

In [ ]:
cuadri_h = np.array(list(cuadruples_h.values()))
triple_h = np.array(list(triples_h.values()))
cuadri_l = np.array(list(cuadruples_l.values()))
triple_l = np.array(list(triples_l.values()))
cuadri_b = np.array(list(cuadruples_b.values()))
triple_b = np.array(list(triples_b.values()))

canales = list(triples_h.keys())

with np.errstate(divide='ignore', invalid='ignore'):
    efi_h = cuadri_h/(triple_h+cuadri_h)
    efi_l = cuadri_l/(triple_l+cuadri_l)
    efi_b = cuadri_b/(triple_b+cuadri_b)

    err_efi_h = np.sqrt(cuadri_h*((triple_h/(cuadri_h+triple_h)**2))**2 + triple_h*((cuadri_h/(cuadri_h+triple_h)**2))**2)
    err_efi_l = np.sqrt(cuadri_l*((triple_l/(cuadri_l+triple_l)**2))**2 + triple_l*((cuadri_l/(cuadri_l+triple_l)**2))**2)
    err_efi_b = np.sqrt(cuadri_b*((triple_b/(cuadri_b+triple_b)**2))**2 + triple_b*((cuadri_b/(cuadri_b+triple_b)**2))**2)

    fig1, ax1 = plt.subplots(1, 1, figsize=(12, 6))
    fig1, ax2 = plt.subplots(1, 1, figsize=(12, 6))
    fig1, ax3 = plt.subplots(1, 1, figsize=(12, 6))

In [ ]:
ax1.errorbar(canales, efi_h, yerr=err_efi_h, fmt='o', color='blue', ecolor='red', capsize=3, label=f"Conteos = {len(n_counts_h)}")
ax1.set_title("Eficiencia para HG")
ax1.set_xlabel("Canales")
ax1.set_ylabel("Eficiencia")
ax1.legend()

ax2.errorbar(canales, efi_l, yerr=err_efi_l, fmt='o', color='blue', ecolor='red', capsize=3, label=f"Conteos = {len(n_counts_l)}")
ax2.set_title("Eficiencia para LG")
ax2.set_xlabel("Canales")
ax2.set_ylabel("Eficiencia")
ax2.legend()

ax3.errorbar(canales, efi_b, yerr=err_efi_b, fmt='o', color='blue', ecolor='red', capsize=3, label=f"Conteos = {len(n_counts_b)}")
ax3.set_title("Eficiencia para HG/LG")
ax3.set_xlabel("Canales")
ax3.set_ylabel("Eficiencia")
ax3.legend()
print(len(n_counts_h),len(n_counts_l), len(n_counts_b))